# RAG for PubMed 

## Problem:
Researchers and healthcare professionals often struggle to quickly find precise, up-to-date scientific information across the massive and growing PubMed database. The manual search process is time-consuming and often results in information overload or missed relevant studies.

## Objective:
Develop a Retrieval-Augmented Generation (RAG) system for PubMed that enables users to input natural language queries and receive concise, accurate, and up-to-date synthesized responses based on the most relevant peer-reviewed literature, significantly reducing research time and improving decision-making quality.

# Import 

In [7]:
# import 
from Bio import Entrez
import psycopg2
from db.db_utils import store_query, get_connection
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from openai import OpenAI


# Data Acquisition

In [6]:
# email for access
Entrez.email = "rithvik0427@gmail.com"  # delete this later 
email = Entrez.email

# Step 2B: Define your search query
search_query = "Keratoconus"  # modifiable

# Step 2C: Search for articles
handle = Entrez.esearch(db="pubmed", term=search_query, retmax=10)  
record = Entrez.read(handle)
id_list = record["IdList"]
print(f"Found {len(id_list)} articles!")

# Step 2D: Fetch the article abstracts
handle = Entrez.efetch(db="pubmed", id=id_list, rettype="abstract", retmode="text")
abstracts_text = handle.read()

# Step 2E: Print to check
print(abstracts_text)


Found 10 articles!
1. Cornea. 2025 Apr 30. doi: 10.1097/ICO.0000000000003882. Online ahead of print.

Intrastromal Keratoplasty Using Biosynthetic Corneas Versus Human Donor Corneas 
in Advanced Keratoconus.

Bansal V(1), Bari A(1), Rafat M(2), Sinha R(1), Agarwal T(1), Maharana PK(1), 
Angmo D(1), Dada T(1), Titiyal JS(1), Tabe S(2), Miron A(2), Sharma N(1).

Author information:
(1)Dr Rajendra Prasad Centre for Ophthalmic Sciences, All India Institute of 
Medical Sciences, New Delhi, India; and.
(2)Linkocare Life Sciences AB, Stockholm, Sweden.

PURPOSE: The objective of this study was to compare the clinical outcomes of 
intrastromal keratoplasty using biosynthetic corneas with human donor corneas in 
cases of advanced keratoconus.
METHODS: A prospective nonblinded randomized controlled study was conducted at a 
tertiary eye center in India. Treatment-naïve patients with advanced keratoconus 
aged 18 years or older were included. Twenty eyes were randomized into 2 
groups-group 1 (hu

In [7]:
store_query(email=email, topic=search_query, data=abstracts_text,query="test"  )

✅ Query stored.


# chunking the text

In [8]:
# Step 3: Use LangChain's production-grade chunker
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # ~500 characters, you can tune this
    chunk_overlap=50,  # overlap to preserve context
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)

chunks = text_splitter.create_documents([abstracts_text])
chunk_texts = [doc.page_content for doc in chunks]

print(f"Chunks created: {len(chunk_texts)}")
print("Sample chunk:\n", chunk_texts[0])


Chunks created: 32
Sample chunk:
 1. Cornea. 2025 Apr 30. doi: 10.1097/ICO.0000000000003882. Online ahead of print.

Intrastromal Keratoplasty Using Biosynthetic Corneas Versus Human Donor Corneas 
in Advanced Keratoconus.

Bansal V(1), Bari A(1), Rafat M(2), Sinha R(1), Agarwal T(1), Maharana PK(1), 
Angmo D(1), Dada T(1), Titiyal JS(1), Tabe S(2), Miron A(2), Sharma N(1).

Author information:
(1)Dr Rajendra Prasad Centre for Ophthalmic Sciences, All India Institute of 
Medical Sciences, New Delhi, India; and.
(2)Linkocare Life Sciences AB, Stockholm, Sweden.


# Embedding the chunks 

In [9]:
# Load biomedical embedding model
model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")

# Embed all chunks
embeddings = model.encode(chunk_texts, show_progress_bar=True)

# Check
print(f"Generated {len(embeddings)} embeddings, each of size {len(embeddings[0])}")


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.82s/it]

Generated 32 embeddings, each of size 768


# Storing the chunks in a vector DB (chromadb)

# 

In [16]:
# Use the new PersistentClient API (auto-persists to ./chroma_db)
client = chromadb.PersistentClient(
    path="./chroma_db",           # where to store on disk
    settings=Settings(),          # default settings
    tenant=DEFAULT_TENANT,        # usually leave as DEFAULT_TENANT
    database=DEFAULT_DATABASE     # usually leave as DEFAULT_DATABASE
)

# Create or get a collection
collection = client.get_or_create_collection(name="pubmed_collection")

# Prepare IDs and metadata
ids = [str(i) for i in range(len(chunk_texts))]
metadatas = [{"source": "pubmed", "chunk_index": i} for i in range(len(chunk_texts))]

# Add documents, embeddings, and metadata to the collection
collection.add(
    ids=ids,
    documents=chunk_texts,
    embeddings=embeddings.tolist(),
    metadatas=metadatas
)


print(f"Stored {collection.count()} chunks in ChromaDB.")

Stored 32 chunks in ChromaDB.


# Retrival Augmented Generation 

In [8]:
# Step 6A: Load embedding model and Chroma collection
model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
client = chromadb.PersistentClient(
    path="./chroma_db",           # where to store on disk
    settings=Settings(),          # default settings
    tenant=DEFAULT_TENANT,        # usually leave as DEFAULT_TENANT
    database=DEFAULT_DATABASE     # usually leave as DEFAULT_DATABASE
)
collection = client.get_collection(name="pubmed_collection")

# Step 6B: Define retrieval function
def retrieve_chunks(query, k=10):
    q_emb = model.encode([query]).tolist()
    result = collection.query(
        query_embeddings=q_emb,
        n_results=k,
        include=["documents","metadatas","distances"]
    )
    for idx, (doc, meta, dist) in enumerate(zip(result["documents"][0], result["metadatas"][0], result["distances"][0]), 1):
        print(f"Result {idx}: [score={dist:.4f}] {meta}")
        print(doc, "\n")
    return result


def answer_question(query, k=10):
    retrieval = retrieve_chunks(query, k)
    context = "\n\n".join(retrieval["documents"][0])
    prompt = f"""You are a medical research assistant. Use the following PubMed abstracts to answer the question.\n\n{context}\n\nQuestion: {query}\nAnswer:"""
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": "You are a helpful medical research assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    print("Answer:\n", response.choices[0].message.content)

# Example usage
answer_question("What are the most recent keratoconus treatment advancements?", k=5)



Result 1: [score=128.3068] {'source': 'pubmed', 'chunk_index': 27}
PURPOSE OF REVIEW: Keratoconus is one of the leading indications for corneal 
transplantation surgery. Although the surgical success rate is high, the 
long-term outcomes do not match other solid organ transplantation procedures. 
The availability of minimally invasive techniques has led to a reduction in the 
number of corneal transplantations in addition to improved visual outcomes in 
patients with keratoconus.
RECENT FINDINGS: Collagen crosslinking slows down or stops disease progression 
in keratoconus. Customized contact lenses provide patient comfort and excellent 
visual outcomes. Bowman's layer onlay grafts have demonstrated promising 
outcomes in keratoconus patients. Corneal regeneration approaches provide hope 
as the next generation treatment option.
SUMMARY: Minimally invasive treatments for keratoconus patients enable 
alternatives to corneal transplantation. This review summarizes some of the 
commonly u

# Testing

In [30]:
# Step 7: Test the QA pipeline with sample questions

sample_questions = [
    "what does the latest research about keratoconus tell us?",
]

for q in sample_questions:
    print(f"\n=== Question: {q} ===")
    answer_question(q, k=10)


=== Question: what does the latest research about keratoconus tell us? ===
Result 1: [score=133.9019] {'source': 'pubmed', 'chunk_index': 7}
PURPOSE: To analyze the corneal morphology of keratoconus using Pentacam, 
Sirius, and CASIA2.
METHODS: Patients diagnosed with keratoconus at the Eye and ENT Hospital of 
Fudan University (January-November 2022) were included. Corneal refraction, 
astigmatism, thickness, and elevation parameters were measured using Pentacam, 
Sirius, and CASIA2. The measurement differences, agreement, and the sensitivity 
and specificity of each instrument for diagnosing keratoconus was evaluated. P 
<0.05 was considered statistically significant.
RESULTS: A total of 61 keratoconus eyes and 48 healthy myopic eyes were 
included. The Ks and Kf measured using Sirius were the highest (p <0.0001). The 
thinnest corneal thickness measured using Pentacam was the thickest (p <0.0001); 
however, the significance disappears in the Kmax ≤48D keratoconus group. When 
the b

# storing the data

In [ ]:
# adding a column to the db called retrived chuncks, query, and answer
def store_retrieved_data(email, topic, query, retrieved_chunks, answer):
    conn =  get_connection()
    cursor = conn.cursor()
    cursor.execute(
        """
        UPDATE user_data
        SET retrieved_chunks = %s, answer = %s
        WHERE email = %s AND topic = %s AND query = %s
        """,
        (retrieved_chunks, answer, email, topic, query)
    )
    conn.commit()
    cursor.close()
    conn.close()

# final full usage 

In [ ]:
# final full pipeline implementation
def full_pipeline(email, search_query, question):
    # Step 1: Fetch articles
    handle = Entrez.esearch(db="pubmed", term=search_query, retmax=10)
    record = Entrez.read(handle)
    id_list = record["IdList"]
    print(f"Found {len(id_list)} articles!")

    # Step 2: Fetch the article abstracts
    handle = Entrez.efetch(db="pubmed", id=id_list, rettype="abstract", retmode="text")
    abstracts_text = handle.read()

    # Step 3: Store query in the database
    store_query(email=email, topic=search_query, data=abstracts_text, query=question)

    # Step 4: Chunk the text
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", "!", "?", " ", ""]
    )
    chunks = text_splitter.create_documents([abstracts_text])
    chunk_texts = [doc.page_content for doc in chunks]

    # Step 5: Embed the chunks
    model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
    embeddings = model.encode(chunk_texts, show_progress_bar=True)

    # Step 6: Store in ChromaDB
    client = chromadb.PersistentClient(
        path="./chroma_db",
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE
    )
    collection = client.get_or_create_collection(name="pubmed_collection")

    ids = [str(i) for i in range(len(chunk_texts))]
    metadatas = [{"source": "pubmed", "chunk_index": i} for i in range(len(chunk_texts))]

    collection.add(
        ids=ids,
        documents=chunk_texts,
        embeddings=embeddings.tolist(),
        metadatas=metadatas
    )

    print(f"Stored {collection.count()} chunks in ChromaDB.")

    # Step 7: Answer the question
    answer_question(question, k=5)
    
    # Step 8: Store retrieved data in the database
    retrieved_chunks = "\n\n".join(chunk_texts)  # or however you want to format it

# modifying the DB to log the conversations better 


In [1]:

import psycopg2, os
from dotenv import load_dotenv
load_dotenv("env_variables.env")

DDL = """
CREATE EXTENSION IF NOT EXISTS pgcrypto;
CREATE TABLE IF NOT EXISTS conversations (
    convo_id        UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    email           TEXT,
    topic           TEXT,
    conversation    JSONB NOT NULL,
    created_at      TIMESTAMPTZ NOT NULL DEFAULT NOW()
);
"""

conn = psycopg2.connect(
    host=os.environ["DB_HOST"],
    port=os.environ.get("DB_PORT", 5432),
    dbname=os.environ["DB_NAME"],
    user=os.environ["DB_USER"],
    password=os.environ["DB_PASSWORD"],
)
with conn, conn.cursor() as cur:
    cur.execute(DDL)
print("✅  conversations table ready.")


✅  conversations table ready.


In [4]:
"""
pubmed_rag_chat.py  –  CLI PubMed RAG chat (no Postgres)

Features
--------
1. Fetch PubMed abstracts for a user‑supplied topic
2. Split into chunks, embed with BioBERT
3. Store/retrieve in a temporary Chroma collection
4. Chat with OpenAI (GPT‑4‑turbo default) using Retrieval‑Augmented Generation
5. Clean up vector store on exit

Requirements
------------
pip install biopython chromadb sentence-transformers langchain openai python-dotenv
"""

import os, uuid, logging
from typing import List, Dict, Any, Optional, Union

from dotenv import load_dotenv
from Bio import Entrez
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from openai import OpenAI

# ──────────────────────────────  configuration  ──────────────────────────────

load_dotenv("env_variables.env")          # expect OPENAI_API_KEY, ENTrez email, ...

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

# ──────────────────────────────  utilities  ──────────────────────────────


def fetch_pubmed_abstracts(
    email: str,
    topic: str,
    db: str = "pubmed",
    retmax: int = 10,
) -> str:
    """Fetch abstracts for *topic* and return them as one long string."""
    Entrez.email = email
    handle = Entrez.esearch(db=db, term=topic, retmax=retmax)
    record = Entrez.read(handle)
    id_list = record["IdList"]
    logger.info(f"Found {len(id_list)} articles for '{topic}'")

    handle = Entrez.efetch(db=db, id=id_list, rettype="abstract", retmode="text")
    return handle.read()


def chunk_text(
    text: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 50,
    separators: Optional[List[str]] = None,
) -> List[str]:
    """Split *text* into overlapping chunks."""
    if separators is None:
        separators = ["\n\n", "\n", ".", "!", "?", " ", ""]
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, separators=separators
    )
    docs = splitter.create_documents([text])
    chunks = [d.page_content for d in docs]
    logger.info(f"Chunks created: {len(chunks)}")
    return chunks


def embed_chunks(
    model: SentenceTransformer,
    chunks: List[str],
) -> List[List[float]]:
    """Return list of embeddings (lists of floats)."""
    emb = model.encode(chunks, show_progress_bar=True)
    logger.info(f"Generated {len(emb)} embeddings, size {emb.shape[1]}")
    return emb.tolist()


# ──────────────────────────────  Chroma manager  ──────────────────────────────


class ChromaDBManager:
    """Owns one collection (unique per chat)."""

    def __init__(self, collection_name: str, db_path: str = "./chroma_db"):
        self.collection_name = collection_name
        self.client = chromadb.PersistentClient(
            path=db_path,
            settings=Settings(),
            tenant=DEFAULT_TENANT,
            database=DEFAULT_DATABASE,
        )

    def collection(self):
        return self.client.get_or_create_collection(name=self.collection_name)

    def add_chunks(self, chunks: List[str], embeddings: List[List[float]]):
        ids = [str(uuid.uuid4()) for _ in chunks]
        self.collection().add(ids=ids, documents=chunks, embeddings=embeddings)

    def query(self, query_text: str, model: SentenceTransformer, k: int = 10):
        q_emb = model.encode([query_text]).tolist()
        return self.collection().query(
            query_embeddings=q_emb,
            n_results=k,
            include=["documents", "distances"],
        )

    def drop(self):
        self.client.delete_collection(self.collection_name)


# ──────────────────────────────  RAG orchestrator  ────────────────────────────


class PubMedRAG:
    """Wrapper that indexes PubMed text then answers questions."""

    def __init__(
        self,
        chroma_mgr: ChromaDBManager,
        embedding_model_name: str = "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb",
        openai_model: str = "gpt-4.1-nano",
        temperature: float = 0.2,
    ):
        self.chroma_mgr = chroma_mgr
        self.model_name = embedding_model_name
        self.embedder = SentenceTransformer(embedding_model_name)
        self.openai = OpenAI()
        self.llm_model = openai_model
        self.temperature = temperature

    # --------------------------------------------------------------------- #
    # indexing                                                               #
    # --------------------------------------------------------------------- #
    def index_topic(self, email: str, topic: str, retmax: int = 10):
        abstracts = fetch_pubmed_abstracts(email, topic, retmax=retmax)
        chunks = chunk_text(abstracts)
        embeddings = embed_chunks(self.embedder, chunks)
        self.chroma_mgr.add_chunks(chunks, embeddings)

    # --------------------------------------------------------------------- #
    # inference                                                              #
    # --------------------------------------------------------------------- #
    def answer(self, question: str, k: int = 10) -> Dict[str, Any]:
        retrieval = self.chroma_mgr.query(question, self.embedder, k=k)
        context = "\n\n".join(retrieval["documents"][0])

        prompt = (
            "You are a helpful medical research assistant.\n\n"
            f"Context:\n{context}\n\n"
            f"Question: {question}\nAnswer:"
        )

        resp = self.openai.chat.completions.create(
            model=self.llm_model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
        )
        answer_text = resp.choices[0].message.content.strip()

        return {
            "answer": answer_text,
            "retrieval": retrieval,
            "chunks_text": context,
        }


# ──────────────────────────────  chat session  ────────────────────────────────


class ChatSession:
    """High‑level chat interface (keeps history in memory only)."""

    def __init__(self, email: str, topic: str, retmax: int = 10):
        self.id = str(uuid.uuid4())
        self.email = email
        self.topic = topic

        # make a temp collection unique to this chat
        self.chroma_mgr = ChromaDBManager(collection_name=f"pubmed_{self.id}")
        self.rag = PubMedRAG(self.chroma_mgr)

        # fetch & index abstracts
        self.rag.index_topic(email, topic, retmax=retmax)
        logger.info("✅  Abstracts indexed.")

        self.turns: List[Dict[str, Any]] = [
            {"role": "system", "content": "You are a helpful medical research assistant."}
        ]

    def ask(self, question: str, k: int = 10) -> str:
        self.turns.append({"role": "user", "content": question})

        result = self.rag.answer(question, k=k)
        answer_text = result["answer"]

        self.turns.append(
            {
                "role": "assistant",
                "content": answer_text,
                "chunks_text": result["chunks_text"],
            }
        )
        return answer_text

    def close(self):
        """Drop vector store to keep disk tidy."""
        self.chroma_mgr.drop()


# ──────────────────────────────  command‑line driver  ─────────────────────────


def main():
    email = input("NCBI Entrez email: ").strip() or os.getenv("ENTREZ_EMAIL", "")
    if not email:
        print("❌  Need an email for Entrez API. Set ENTREZ_EMAIL or type it now.")
        return

    topic = input("PubMed search topic: ").strip()
    if not topic:
        print("❌  Topic cannot be empty.")
        return

    session = ChatSession(email, topic, retmax=10)

    try:
        while True:
            question = input("\nYou: ").strip()
            if question.lower() in {"quit", "exit"}:
                break
            answer = session.ask(question, k=10)
            print(f"\nAssistant: {answer}")
    finally:
        session.close()
        print("🔚  Session ended (vector store removed).")


if __name__ == "__main__":
    main()


2025-05-09 14:24:19,452 - INFO - Use pytorch device_name: mps
2025-05-09 14:24:19,453 - INFO - Load pretrained SentenceTransformer: pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb
2025-05-09 14:24:21,260 - INFO - Found 0 articles for 'keretoconus'
2025-05-09 14:24:21,545 - INFO - Chunks created: 1
Batches: 100%|██████████| 1/1 [00:00<00:00, 33.86it/s]
2025-05-09 14:24:21,582 - INFO - Generated 1 embeddings, size 768
2025-05-09 14:24:21,642 - INFO - ✅  Abstracts indexed.
Batches: 100%|██████████| 1/1 [00:00<00:00, 12.66it/s]
2025-05-09 14:26:02,719 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Assistant: Based on current research, scleral lenses are considered an effective correction method for certain visual impairments, especially irregular corneas and keratoconus. However, depending on the specific condition and patient needs, alternative correction methods may be preferable. These include:

1. **Rigid Gas Permeable (RGP) Contact Lenses:** Often used for irregular corneas, providing good visual acuity and corneal protection.

2. **Soft Contact Lenses:** Suitable for general refractive errors like myopia, hyperopia, and astigmatism, especially when comfort is a priority.

3. **Orthokeratology (Ortho-K):** Specially designed rigid lenses worn overnight to temporarily reshape the cornea and correct myopia.

4. **Refractive Surgery (e.g., LASIK, PRK):** Surgical options that permanently alter corneal shape for vision correction.

5. **Implantable Collamer Lenses (ICL):** Surgically implanted lenses for high refractive errors or when corneal surgery isn't suitable.

The choic

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.68it/s]
2025-05-09 14:26:17,924 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Assistant: It appears that the input provided is incomplete or missing specific details. Could you please clarify your question or provide additional information so I can assist you effectively?


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.48it/s]
2025-05-09 14:26:30,623 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Assistant: It appears that the supplied ID and parameter are empty. Could you please provide the specific ID or parameter you would like information about? This will help me assist you more effectively.


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.49it/s]
2025-05-09 14:26:37,688 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Assistant: It appears that the provided input is incomplete or contains placeholder text ("Supplied+id+parameter+is+empty"). Could you please clarify your question or provide more details so I can assist you effectively?
🔚  Session ended (vector store removed).
